## Try it online with Colab Notebooks!

见子文件夹。

## Basic Usage: Training, Saving, Loading

这个示例展示了如何在Lunar Lander环境上训练、保存和加载一个DQN模型的基本用法。首先，你需要确保安装了必要的包，包括`gymnasium`和`stable_baselines3`，以及环境所需的`box2d`。

### 安装必要的包

安装`box2d`环境的依赖项：

```bash
apt install swig
pip install box2d box2d-kengz
```

### 训练模型

下面是创建环境、实例化模型、训练模型、保存模型以及重新加载模型的代码示例：

```python
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy

# 创建环境
env = gym.make("LunarLander-v2", render_mode="rgb_array")

# 实例化智能体
model = DQN("MlpPolicy", env, verbose=1)
# 训练智能体并显示进度条
model.learn(total_timesteps=int(2e5), progress_bar=True)
# 保存智能体
model.save("dqn_lunar")
del model  # 删除训练好的模型以演示加载过程

# 加载训练好的智能体
model = DQN.load("dqn_lunar", env=env)

# 评估智能体
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)

# 观看训练好的智能体
vec_env = model.get_env()
obs = vec_env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")
```

### 注意事项

- 加载模型时，请直接使用`DQN.load`方法而不是先实例化`DQN`然后再调用`load`。前者将从头创建模型，而后者（`model.load`）不是原地操作，可能无法如预期工作。
- 如果你使用了修改奖励的环境包装器进行训练，评估时也会反映出来。如果你想用原始奖励进行评估，可以在其他包装器之前使用`Monitor`包装器来包装环境。

这个过程演示了在强化学习中训练、保存和重新加载模型的典型流程，可以帮助你在实际项目中快速上手并有效地管理你的RL模型。

## Multiprocessing: Unleashing the Power of Vectorized Environments

In [ ]:
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed

def make_env(env_id: str, rank: int, seed: int = 0):
    """
    Utility function for multiprocessed env.

    :param env_id: the environment ID
    :param num_env: the number of environments you wish to have in subprocesses
    :param seed: the inital seed for RNG
    :param rank: index of the subprocess
    """
    def _init():
        env = gym.make(env_id, render_mode="human")
        env.reset(seed=seed + rank)
        return env
    set_random_seed(seed)
    return _init

if __name__ == "__main__":
    env_id = "CartPole-v1"
    num_cpu = 4  # Number of processes to use
    # Create the vectorized environment
    vec_env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])

    # Stable Baselines provides you with make_vec_env() helper
    # which does exactly the previous steps for you.
    # You can choose between `DummyVecEnv` (usually faster) and `SubprocVecEnv`
    # env = make_vec_env(env_id, n_envs=num_cpu, seed=0, vec_env_cls=SubprocVecEnv)

    model = PPO("MlpPolicy", vec_env, verbose=1)
    model.learn(total_timesteps=25_000)

    obs = vec_env.reset()
    for _ in range(1000):
        action, _states = model.predict(obs)
        obs, rewards, dones, info = vec_env.step(action)
        vec_env.render()

这段代码演示了如何使用Stable Baselines3和多进程环境来训练强化学习模型。我们将分步骤详细解释每个部分，以便你作为RL新手能更好地理解。

### 引入必要的库

首先，代码引入了必要的库和函数：

```python
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed
```

- `gymnasium`是一个提供各种强化学习环境的库。
- `stable_baselines3`是一个提供高级强化学习算法的库。
- `DummyVecEnv`和`SubprocVecEnv`是用于创建向量化环境的类，向量化环境可以让我们并行地运行多个环境实例，从而加速训练过程。
- `make_vec_env`是一个辅助函数，用于简化向量化环境的创建过程。
- `set_random_seed`用于设置随机种子，以确保实验的可重复性。

### 创建向量化环境

```python
def make_env(env_id: str, rank: int, seed: int = 0):
    def _init():
        env = gym.make(env_id, render_mode="human")
        env.reset(seed=seed + rank)
        return env
    set_random_seed(seed)
    return _init
```

- `make_env`是一个工厂函数，用于创建环境实例。它接受环境ID(`env_id`)，子进程索引(`rank`)，和随机种子(`seed`)作为参数。
- 每个环境通过增加`rank`值来有不同的随机种子，这样每个环境实例都是不同的，有助于提高训练过程的多样性。

### 多进程环境

```python
if __name__ == "__main__":
    env_id = "CartPole-v1"
    num_cpu = 4  # Number of processes to use
    vec_env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])
```

- 这段代码初始化了一个多进程环境`SubprocVecEnv`，其中包含了`num_cpu`个`CartPole-v1`环境的实例。
- `SubprocVecEnv`使用子进程来运行每个环境实例，可以有效地利用多核CPU。

### 训练模型

```python
model = PPO("MlpPolicy", vec_env, verbose=1)
model.learn(total_timesteps=25_000)
```

- 使用PPO算法和多层感知器（`MlpPolicy`）策略在向量化环境上训练模型。
- `total_timesteps=25_000`指定了训练的总时间步数。

### 评估和渲染

```python
obs = vec_env.reset()
for _ in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render()
```

- 这段代码循环运行模型，以评估训练后的性能。
- 在每个时间步，模型根据当前观察`obs`预测动作，然后在环境上执行这个动作，并渲染结果。

### 关于多进程

- 使用`SubprocVecEnv`可以显著加速训练过程，特别是在需要大量时间步来学习的环境中。
- 但是，渲染多进程环境可能会有一些问题，因为每个环境实例都在不同的进程中运行。在实际使用中，通常在单个环境上进行渲染以进行调试和评估。

希望这个详细

解释能帮助你理解如何使用Stable Baselines3和多进程环境进行强化学习模型的训练。如果你有任何疑问，随时提问！

## Multiprocessing with off-policy algorithms


In [ ]:
import gymnasium as gym

from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env

vec_env = make_vec_env("Pendulum-v0", n_envs=4, seed=0)

# We collect 4 transitions per call to `ènv.step()`
# and performs 2 gradient steps per call to `ènv.step()`
# if gradient_steps=-1, then we would do 4 gradients steps per call to `ènv.step()`
model = SAC("MlpPolicy", vec_env, train_freq=1, gradient_steps=2, verbose=1)
model.learn(total_timesteps=10_000)


在使用离策略（off-policy）算法与多环境（multiprocessing）时，特别是在Stable Baselines3库中使用SAC（Soft Actor-Critic）算法时，有一些重要的配置需要注意。这段代码展示了如何在`Pendulum-v0`环境上使用SAC算法，同时运用四个并行环境加速训练过程。以下是对代码的详细解释：

### 创建向量化环境

```python
vec_env = make_vec_env("Pendulum-v0", n_envs=4, seed=0)
```

- `make_vec_env`是一个辅助函数，用于简化多环境实例的创建过程。这里创建了`Pendulum-v0`环境的四个实例，`n_envs=4`表示环境的数量，`seed=0`确保实验的可重复性。

### 配置SAC模型

```python
model = SAC("MlpPolicy", vec_env, train_freq=1, gradient_steps=2, verbose=1)
```

- 这行代码实例化了一个SAC模型，使用了多层感知器（`MlpPolicy`）策略。
- `train_freq=1`表示每个环境步骤（`env.step()`）后进行训练的频率。这里设置为1，意味着每进行一次环境步骤，就会触发一次训练。
- `gradient_steps=2`指定了每次训练时执行的梯度更新步数。这里设置为2，表示每次训练会执行两个梯度更新步骤。

### 关于`gradient_steps`的注意事项

- 当使用多环境时，`gradient_steps`参数的设置尤为重要。默认情况下，如果你不调整`gradient_steps`参数，可能会导致样本利用率不高或训练效率低下。
- 将`gradient_steps`设置为`-1`可以确保执行与收集到的转换（transitions）数量相同的梯度更新步骤。这通常是墙钟时间（wall-clock time）和样本效率（sample efficiency）之间的折中。在这个例子中，我们手动设置`gradient_steps=2`，以便每次调用`env.step()`时执行两个梯度更新步骤。如果环境是四个并行实例，`gradient_steps=-1`将会导致每次`env.step()`调用执行四个梯度更新步骤。

### 训练模型

```python
model.learn(total_timesteps=10_000)
```

- 调用`learn`方法开始训练过程，`total_timesteps=10_000`指定了训练的总时间步数。

### 结论

这个示例展示了如何在使用离策略算法和多环境配置时正确设置`gradient_steps`参数，以平衡训练效率和样本效率。通过适当配置，可以充分利用多环境带来的并行处理优势，加速训练过程，同时保持算法的效能。

## Dict Observations

在强化学习中，有时我们遇到的环境会产生不同类型的观察（观测数据），例如，一个相机捕获的图像和一组伺服传感器数据（如旋转角度）。这种情况下，观察空间可能是字典（Dict）形式，其中包含不同类型的数据。Stable Baselines3通过提供`SimpleMultiObsEnv`环境作为示例，支持字典观察空间的环境。

### 使用字典观察空间

在`SimpleMultiObsEnv`环境中，每个单元的观察以字典形式返回，包含向量观察和图像观察。这是处理复合观察空间的一个示例，非常适用于需要处理多种类型观测数据的场景。

### 示例代码解释

```python
from stable_baselines3 import PPO
from stable_baselines3.common.envs import SimpleMultiObsEnv

# 使用SimpleMultiObsEnv环境，它提供字典形式的观察
env = SimpleMultiObsEnv(random_start=False)

# 使用PPO算法和MultiInputPolicy策略来处理多输入的观察空间
model = PPO("MultiInputPolicy", env, verbose=1)

# 训练模型
model.learn(total_timesteps=100_000)
```

- `SimpleMultiObsEnv(random_start=False)`创建了一个简单的网格世界环境，观察返回为字典形式。`random_start=False`参数指定了环境初始化的方式。
- `PPO("MultiInputPolicy", env, verbose=1)`实例化了一个PPO模型，使用`MultiInputPolicy`策略来处理字典形式的多输入观察。这表明模型能够处理包含多种数据类型的复杂观察空间。
- `model.learn(total_timesteps=100_000)`开始训练过程，总共进行100,000时间步的训练。

### 字典观察空间的处理

在使用字典观察空间时，重要的是要使用一个能够处理多种输入的策略或模型架构。`MultiInputPolicy`是Stable Baselines3为这种情况提供的策略之一，它能够接受并处理多种类型的观察输入。这对于那些需要结合来自不同传感器的数据进行决策的应用场景非常有用。

通过这种方式，你可以构建更复杂、更接近现实世界应用的强化学习模型，有效地处理多种类型的观察数据，从而提高模型的适用性和效能。

## Callbacks: Monitoring Training

In [ ]:
import os

import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt

from stable_baselines3 import TD3
from stable_baselines3.common import results_plotter
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import BaseCallback


class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq:
    :param log_dir: Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: Verbosity level: 0 for no output, 1 for info messages, 2 for debug messages
    """
    def __init__(self, check_freq: int, log_dir: str, verbose: int = 1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, "best_model")
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

          # Retrieve training reward
          x, y = ts2xy(load_results(self.log_dir), "timesteps")
          if len(x) > 0:
              # Mean training reward over the last 100 episodes
              mean_reward = np.mean(y[-100:])
              if self.verbose >= 1:
                print(f"Num timesteps: {self.num_timesteps}")
                print(f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}")

              # New best model, you could save the agent here
              if mean_reward > self.best_mean_reward:
                  self.best_mean_reward = mean_reward
                  # Example for saving best model
                  if self.verbose >= 1:
                    print(f"Saving new best model to {self.save_path}")
                  self.model.save(self.save_path)

        return True

# Create log dir
log_dir = "tmp/"
os.makedirs(log_dir, exist_ok=True)

# Create and wrap the environment
env = gym.make("LunarLanderContinuous-v2")
env = Monitor(env, log_dir)

# Add some action noise for exploration
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))
# Because we use parameter noise, we should use a MlpPolicy with layer normalization
model = TD3("MlpPolicy", env, action_noise=action_noise, verbose=0)
# Create the callback: check every 1000 steps
callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)
# Train the agent
timesteps = 1e5
model.learn(total_timesteps=int(timesteps), callback=callback)

plot_results([log_dir], timesteps, results_plotter.X_TIMESTEPS, "TD3 LunarLander")
plt.show()

这段代码展示了如何在使用Stable Baselines3进行强化学习训练时，通过自定义回调（Callback）来监控训练进度，并在达到最佳训练奖励时保存模型。以下是对代码的详细解释：

### 自定义回调类

```python
class SaveOnBestTrainingRewardCallback(BaseCallback):
```

- 这个类继承自`BaseCallback`，用于在训练过程中执行特定的任务。这里的任务是监控训练奖励，并在达到新的最高平均奖励时保存模型。

### 回调类的主要方法

- `__init__`方法初始化回调类的实例，设置检查频率（`check_freq`）、日志目录（`log_dir`）和最佳模型的保存路径（`save_path`）。
- `_init_callback`方法用于在需要时创建保存模型的文件夹。
- `_on_step`方法在每个训练步骤调用，用于检查是否需要保存模型。如果当前平均奖励超过之前的最佳平均奖励，则更新最佳奖励并保存当前模型。

### 环境准备和模型训练

```python
env = gym.make("LunarLanderContinuous-v2")
env = Monitor(env, log_dir)
```

- 使用`gym`创建`LunarLanderContinuous-v2`环境，并用`Monitor`包装器包装环境以记录详细的训练日志。

```python
model = TD3("MlpPolicy", env, action_noise=action_noise, verbose=0)
callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)
model.learn(total_timesteps=int(timesteps), callback=callback)
```

- 使用TD3算法和多层感知器策略（`MlpPolicy`）实例化模型，添加动作噪声以增强探索性。
- 实例化自定义回调，并设置检查频率为每1000步检查一次。
- 使用`model.learn`方法开始训练，训练过程中将调用回调来监控训练奖励，并在达到最佳奖励时保存模型。

### 结果可视化

```python
plot_results([log_dir], timesteps, results_plotter.X_TIMESTEPS, "TD3 LunarLander")
plt.show()
```

- 使用`plot_results`函数从日志目录加载训练结果，并绘制训练过程中奖励随时间步的变化。这有助于可视化模型的学习进度。

### 重要注意事项

- 使用回调功能是监控训练进度和自动化训练过程（如保存最佳模型）的有效手段。
- `Monitor`包装器不仅记录每个剧情的奖励，还能让回调函数访问这些数据，以决定是否保存当前的最佳模型。
- 在实际应用中，我们建议使用`EvalCallback`来代替直接使用训练奖励作为性能指标，因为`EvalCallback`通过在独立的评估环境上评估模型来提供更准确的性能衡量。

## Callbacks: Evaluate Agent Performance

In [ ]:
import os
import gymnasium as gym

from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env

env_id = "Pendulum-v1"
n_training_envs = 1
n_eval_envs = 5

# Create log dir where evaluation results will be saved
eval_log_dir = "./eval_logs/"
os.makedirs(eval_log_dir, exist_ok=True)

# Initialize a vectorized training environment with default parameters
train_env = make_vec_env(env_id, n_envs=n_training_envs, seed=0)

# Separate evaluation env, with different parameters passed via env_kwargs
# Eval environments can be vectorized to speed up evaluation.
eval_env = make_vec_env(env_id, n_envs=n_eval_envs, seed=0,
                        env_kwargs={'g':0.7})

# Create callback that evaluates agent for 5 episodes every 500 training environment steps.
# When using multiple training environments, agent will be evaluated every
# eval_freq calls to train_env.step(), thus it will be evaluated every
# (eval_freq * n_envs) training steps. See EvalCallback doc for more information.
eval_callback = EvalCallback(eval_env, best_model_save_path=eval_log_dir,
                              log_path=eval_log_dir, eval_freq=max(500 // n_training_envs, 1),
                              n_eval_episodes=5, deterministic=True,
                              render=False)

model = SAC("MlpPolicy", train_env)
model.learn(5000, callback=eval_callback)

这段代码演示了如何在使用Stable Baselines3进行强化学习训练时，使用`EvalCallback`定期评估智能体在单独测试环境上的性能。以下是对代码的详细解释：

### 初始化和创建日志目录

```python
eval_log_dir = "./eval_logs/"
os.makedirs(eval_log_dir, exist_ok=True)
```

- 创建一个目录来保存评估结果和最佳模型。

### 创建训练和评估环境

```python
train_env = make_vec_env(env_id, n_envs=n_training_envs, seed=0)
eval_env = make_vec_env(env_id, n_envs=n_eval_envs, seed=0, env_kwargs={'g':0.7})
```

- 使用`make_vec_env`函数初始化向量化的训练环境和评估环境。`n_training_envs`和`n_eval_envs`分别设置训练和评估环境的数量。
- `env_kwargs={'g':0.7}`在评估环境中设置了不同的环境参数，这有助于评估智能体在略有不同条件下的性能。

### 设置`EvalCallback`

```python
eval_callback = EvalCallback(eval_env, best_model_save_path=eval_log_dir,
                             log_path=eval_log_dir, eval_freq=max(500 // n_training_envs, 1),
                             n_eval_episodes=5, deterministic=True, render=False)
```

- `EvalCallback`用于在训练过程中定期评估智能体性能，并可选地保存最佳模型。
- `eval_freq`控制评估频率。这里设置为每500个训练环境步骤进行一次评估。当使用多个训练环境时，实际的训练步数会乘以环境数量。
- `n_eval_episodes=5`表示每次评估时，智能体将在测试环境中运行5个完整的剧情。
- `deterministic=True`确保评估时智能体采取确定性行为，以便更准确地衡量其性能。

### 训练模型并使用回调

```python
model = SAC("MlpPolicy", train_env)
model.learn(5000, callback=eval_callback)
```

- 使用SAC算法和多层感知器（`MlpPolicy`）策略初始化模型。
- 调用`model.learn`开始训练，传入总时间步数和之前定义的回调函数。这将在训练过程中按照指定的频率自动评估智能体，并根据评估结果保存最佳模型。

### 重要性

使用`EvalCallback`是一个有效的方法，可以在训练过程中持续监控智能体的性能，并确保你总是有最佳的模型版本可用。这种方法特别适用于长时间训练或在复杂环境中训练智能体的情况，其中智能体的性能可能会随时间显著变化。通过定期评估，你可以更好地理解智能体的学习进度，并在必要时调整训练策略。

## Atari Games

In [ ]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import A2C

# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
vec_env = make_atari_env("PongNoFrameskip-v4", n_envs=4, seed=0)
# Frame-stacking with 4 frames
vec_env = VecFrameStack(vec_env, n_stack=4)

model = A2C("CnnPolicy", vec_env, verbose=1)
model.learn(total_timesteps=25_000)

obs = vec_env.reset()
while True:
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")

训练强化学习（RL）智能体在Atari游戏上是一个非常直接的过程，尤其是当使用Stable Baselines3库时。以下是如何做到这一点的详细步骤解释：

### 安装Atari环境

为了在Atari游戏上训练RL智能体，首先需要安装Atari环境。这可以通过运行以下命令来完成：

```sh
pip install gymnasium[atari,accept-rom-license]
```

这个命令会安装Atari环境以及相应的ROMs。如果你想要安装Stable Baselines3以及其所有可选依赖项（包括Atari环境），你可以使用：

```sh
pip install stable-baselines3[extra]
```

### 创建和预处理Atari环境

```python
vec_env = make_atari_env("PongNoFrameskip-v4", n_envs=4, seed=0)
vec_env = VecFrameStack(vec_env, n_stack=4)
```

- `make_atari_env`辅助函数用于创建和预处理Atari环境。这包括环境的正确初始化以及应用必要的预处理步骤（如帧跳过和灰度转换）。
- `PongNoFrameskip-v4`是游戏的ID，这里我们选择了Atari的Pong游戏。
- `n_envs=4`表示我们同时创建四个环境实例，这有助于加速训练过程。
- `VecFrameStack`用于实现帧堆叠，这是一种常见的预处理技术，可以给智能体提供一定程度的运动信息。这里，`n_stack=4`表示我们将连续四帧堆叠在一起作为每个决策点的观察。

### 训练模型

```python
model = A2C("CnnPolicy", vec_env, verbose=1)
model.learn(total_timesteps=25_000)
```

- 使用A2C算法和卷积神经网络策略（`CnnPolicy`）来训练模型。这里选择的`CnnPolicy`特别适用于处理视觉输入，如Atari游戏中的像素帧。
- `model.learn(total_timesteps=25_000)`开始训练过程，其中`total_timesteps=25_000`指定了训练的总时间步数。

### 观察和交互

```python
obs = vec_env.reset()
while True:
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")
```

- 这段代码展示了如何使用训练好的模型来进行观察和交互。`model.predict(obs, deterministic=False)`根据当前观察生成动作。这里，`deterministic=False`表示动作选择包含一定的随机性，有助于探索。
- `vec_env.step(action)`执行选定的动作并返回新的观察、奖励和完成标志。
- `vec_env.render("human")`用于渲染环境，使我们能够直观地看到智能体在游戏中的表现。

通过上述步骤，你可以在Atari游戏上训练并评估RL智能体，利用Stable Baselines3提供的工具和预处理功能简化整个流程。

## PyBullet: Normalizing input features

In [ ]:
import os
import gymnasium as gym
import pybullet_envs

from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3 import PPO

# Note: pybullet is not compatible yet with Gymnasium
# you might need to use `import rl_zoo3.gym_patches`
# and use gym (not Gymnasium) to instantiate the env
# Alternatively, you can use the MuJoCo equivalent "HalfCheetah-v4"
vec_env = DummyVecEnv([lambda: gym.make("HalfCheetahBulletEnv-v0")])
# Automatically normalize the input features and reward
vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=True,
                   clip_obs=10.)

model = PPO("MlpPolicy", vec_env)
model.learn(total_timesteps=2000)

# Don't forget to save the VecNormalize statistics when saving the agent
log_dir = "/tmp/"
model.save(log_dir + "ppo_halfcheetah")
stats_path = os.path.join(log_dir, "vec_normalize.pkl")
env.save(stats_path)

# To demonstrate loading
del model, vec_env

# Load the saved statistics
vec_env = DummyVecEnv([lambda: gym.make("HalfCheetahBulletEnv-v0")])
vec_env = VecNormalize.load(stats_path, vec_env)
#  do not update them at test time
vec_env.training = False
# reward normalization is not needed at test time
vec_env.norm_reward = False

# Load the agent
model = PPO.load(log_dir + "ppo_halfcheetah", env=vec_env)

这段代码演示了如何在使用PyBullet环境进行强化学习训练时，对输入特征（观察）和奖励进行标准化处理。标准化处理可以帮助RL智能体更有效地学习，特别是在输入特征的规模不一或奖励差异较大的情况下。以下是对代码的详细解释：

### 安装PyBullet

首先，你需要安装PyBullet，这可以通过运行以下命令来完成：

```sh
pip install pybullet
```

### 创建和标准化环境

```python
vec_env = DummyVecEnv([lambda: gym.make("HalfCheetahBulletEnv-v0")])
vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=True, clip_obs=10.)
```

- 使用`DummyVecEnv`创建一个PyBullet环境的实例，这里使用的是`HalfCheetahBulletEnv-v0`环境。
- 通过`VecNormalize`包装器对环境进行包装，以自动标准化观察（输入特征）和奖励。`norm_obs=True`和`norm_reward=True`启用了观察和奖励的标准化，`clip_obs=10.`限制了观察值的最大绝对值，以避免极端值影响训练。

### 训练模型

```python
model = PPO("MlpPolicy", vec_env)
model.learn(total_timesteps=2000)
```

- 使用PPO算法和多层感知器策略（`MlpPolicy`）实例化模型。
- 调用`model.learn`方法开始训练，其中指定了训练的总时间步数。

### 保存模型和环境统计信息

```python
model.save(log_dir + "ppo_halfcheetah")
env.save(stats_path)
```

- 训练完成后，保存模型和包含标准化统计信息的环境。这些统计信息对于后续加载模型和环境以进行评估或继续训练是必要的。

### 加载模型和环境

```python
vec_env = VecNormalize.load(stats_path, vec_env)
vec_env.training = False
vec_env.norm_reward = False
model = PPO.load(log_dir + "ppo_halfcheetah", env=vec_env)
```

- 首先重新创建环境，并使用保存的统计信息加载`VecNormalize`包装器。设置`vec_env.training = False`和`vec_env.norm_reward = False`，以在测试时停止更新统计信息并停用奖励标准化。
- 然后加载之前保存的模型，注意在加载模型时传入经过标准化处理的环境。

### 重要注意事项

- 在训练和测试阶段对观察和奖励进行标准化处理是一种常见且有效的做法，可以帮助提高RL智能体的学习效率和性能。
- 保存和加载`VecNormalize`包装器的统计信息是确保模型能够在加载后以相同的方式处理观察和奖励的关键步骤。
- 当在实际应用中使用标准化包装器时，确保在评估或部署模型之前正确设置包装器的状态（如禁用训练和奖励标准化）。

## Hindsight Experience Replay (HER)

In [ ]:
import gymnasium as gym
import highway_env
import numpy as np

from stable_baselines3 import HerReplayBuffer, SAC, DDPG, TD3
from stable_baselines3.common.noise import NormalActionNoise

env = gym.make("parking-v0")

# Create 4 artificial transitions per real transition
n_sampled_goal = 4

# SAC hyperparams:
model = SAC(
    "MultiInputPolicy",
    env,
    replay_buffer_class=HerReplayBuffer,
    replay_buffer_kwargs=dict(
      n_sampled_goal=n_sampled_goal,
      goal_selection_strategy="future",
    ),
    verbose=1,
    buffer_size=int(1e6),
    learning_rate=1e-3,
    gamma=0.95,
    batch_size=256,
    policy_kwargs=dict(net_arch=[256, 256, 256]),
)

model.learn(int(2e5))
model.save("her_sac_highway")

# Load saved model
# Because it needs access to `env.compute_reward()`
# HER must be loaded with the env
env = gym.make("parking-v0", render_mode="human") # Change the render mode
model = SAC.load("her_sac_highway", env=env)

obs, info = env.reset()

# Evaluate the agent
episode_reward = 0
for _ in range(100):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    episode_reward += reward
    if terminated or truncated or info.get("is_success", False):
        print("Reward:", episode_reward, "Success?", info.get("is_success", False))
        episode_reward = 0.0
        obs, info = env.reset()

这个例子展示了如何使用Hindsight Experience Replay（HER）来训练一个在Highway-Env环境中的强化学习智能体。Highway-Env是由@eleurent开发的，专门用于研究自动驾驶和其他交通相关的决策问题。其中，`parking-v0`环境是一个目标条件下的连续控制任务，目标是控制车辆停在指定的车位上，并且具有适当的朝向。

### Hindsight Experience Replay (HER)

HER是一种针对稀疏奖励环境设计的强化学习算法。在这类环境中，智能体很难通过随机探索找到奖励，这使得学习过程非常缓慢。HER的核心思想是利用每个失败的经验，通过重新设定目标（即便是原本未达成的目标），将这些失败的尝试转化为成功的经验。这种方法显著提高了学习效率，特别是在目标条件任务中。

### Highway-Env环境简介

![Highway Parking Env](https://raw.githubusercontent.com/eleurent/highway-env/gh-media/docs/media/parking-env.gif)

`parking-v0`环境模拟了一个停车任务，智能体控制一辆车，必须在没有碰撞的情况下驾驶到指定的车位并且调整到正确的朝向。这个任务要求智能体学习如何准确地控制车辆的速度和方向，同时考虑到环境中的其他车辆和障碍物。

### 使用HER训练智能体

在这个例子中，使用SAC（Soft Actor-Critic）算法结合HER回放缓冲区来训练智能体。SAC是一种基于梯度的算法，适用于连续动作空间的任务，且由于其熵正则化项，能够鼓励探索。

```python
model = SAC(
    "MultiInputPolicy",
    env,
    replay_buffer_class=HerReplayBuffer,
    replay_buffer_kwargs=dict(
        n_sampled_goal=n_sampled_goal,
        goal_selection_strategy="future",
    ),
    verbose=1,
    buffer_size=int(1e6),
    learning_rate=1e-3,
    gamma=0.95,
    batch_size=256,
    policy_kwargs=dict(net_arch=[256, 256, 256]),
)
```

在这段代码中，`replay_buffer_class=HerReplayBuffer`指定了使用HER回放缓冲区，`goal_selection_strategy="future"`表示采用"未来"策略来选择新的目标，这是HER算法中常用的一种策略。

### 评估智能体性能

智能体训练完成后，通过一系列的测试剧情来评估其性能。在每个剧情中，智能体尝试完成停车任务，并记录获得的总奖励。如果智能体成功停到指定位置，`info`字典中的`"is_success"`键将为`True`。

这个例子展示了如何在具有稀疏奖励和连续动作空间的复杂任务中应用HER算法。通过这种方法，即使在难以通过随机探索获取奖励的环境中，智能体也能有效地学习并提高性能。

## Learning Rate Schedule

In [ ]:
from typing import Callable

from stable_baselines3 import PPO


def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func

# Initial learning rate of 0.001
model = PPO("MlpPolicy", "CartPole-v1", learning_rate=linear_schedule(0.001), verbose=1)
model.learn(total_timesteps=20_000)
# By default, `reset_num_timesteps` is True, in which case the learning rate schedule resets.
# progress_remaining = 1.0 - (num_timesteps / total_timesteps)
model.learn(total_timesteps=10_000, reset_num_timesteps=True)

这个示例展示了如何在使用Stable Baselines3的PPO算法时应用学习率调度策略。学习率调度是训练过程中动态调整学习率的一种方法，它可以根据训练进度来调整学习率的大小。通过这种方式，可以在训练初期使用较高的学习率以快速收敛，在训练后期则减小学习率以避免过度振荡，从而提高训练的稳定性和效率。

### 线性学习率调度

```python
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    def func(progress_remaining: float) -> float:
        return progress_remaining * initial_value
    return func
```

- `linear_schedule`函数定义了一个线性学习率调度，它接受初始学习率`initial_value`作为参数，并返回一个计算当前学习率的函数。这个计算函数将根据剩余进度（`progress_remaining`），从1（训练开始）线性减少到0（训练结束），来动态调整学习率。

### 应用学习率调度到模型

```python
model = PPO("MlpPolicy", "CartPole-v1", learning_rate=linear_schedule(0.001), verbose=1)
```

- 在初始化PPO模型时，通过`learning_rate`参数传入`linear_schedule(0.001)`，即应用了初始学习率为0.001的线性调度策略。这意味着学习率将从0.001开始，随着训练进度的推进而线性减少。

### 训练模型

```python
model.learn(total_timesteps=20_000)
model.learn(total_timesteps=10_000, reset_num_timesteps=True)
```

- 使用`model.learn`方法开始训练，其中`total_timesteps`指定了训练的总步数。
- 如果再次调用`model.learn`继续训练，并且`reset_num_timesteps`参数为`True`（默认值），学习率调度将会重置，即`progress_remaining`重新从1开始计算。

### 重要性

- 学习率调度是一种重要的训练技巧，能够帮助模型更好地收敛并提高最终性能。
- 线性调度是最简单的调度方式之一，但根据具体任务和训练情况，也可以设计更复杂的调度策略，如指数衰减、周期调度等。
- 在实践中，选择和调整合适的学习率调度策略可以根据实验结果来进行，以找到最佳的训练效果。

## Advanced Saving and Loading

In [ ]:
from stable_baselines3 import SAC
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.sac.policies import MlpPolicy

# Create the model and the training environment
model = SAC("MlpPolicy", "Pendulum-v1", verbose=1,
            learning_rate=1e-3)

# train the model
model.learn(total_timesteps=6000)

# save the model
model.save("sac_pendulum")

# the saved model does not contain the replay buffer
loaded_model = SAC.load("sac_pendulum")
print(f"The loaded_model has {loaded_model.replay_buffer.size()} transitions in its buffer")

# now save the replay buffer too
model.save_replay_buffer("sac_replay_buffer")

# load it into the loaded_model
loaded_model.load_replay_buffer("sac_replay_buffer")

# now the loaded replay is not empty anymore
print(f"The loaded_model has {loaded_model.replay_buffer.size()} transitions in its buffer")

# Save the policy independently from the model
# Note: if you don't save the complete model with `model.save()`
# you cannot continue training afterward
policy = model.policy
policy.save("sac_policy_pendulum")

# Retrieve the environment
env = model.get_env()

# Evaluate the policy
mean_reward, std_reward = evaluate_policy(policy, env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

# Load the policy independently from the model
saved_policy = MlpPolicy.load("sac_policy_pendulum")

# Evaluate the loaded policy
mean_reward, std_reward = evaluate_policy(saved_policy, env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

这个例子演示了在使用Stable Baselines3进行强化学习训练时，如何独立于模型保存和加载策略（Policy），以及如何单独保存和加载回放缓冲区（Replay Buffer）。下面是对代码的详细解释：

### 训练和保存模型

首先，创建一个SAC模型并在`Pendulum-v1`环境上训练：

```python
model = SAC("MlpPolicy", "Pendulum-v1", verbose=1, learning_rate=1e-3)
model.learn(total_timesteps=6000)
model.save("sac_pendulum")
```

这段代码训练了一个SAC模型，并使用`model.save`方法保存了模型。默认情况下，保存的模型不包括回放缓冲区，以节省磁盘空间。

### 保存和加载回放缓冲区

```python
model.save_replay_buffer("sac_replay_buffer")
loaded_model = SAC.load("sac_pendulum")
loaded_model.load_replay_buffer("sac_replay_buffer")
```

为了保存和加载回放缓冲区，使用`model.save_replay_buffer`和`loaded_model.load_replay_buffer`方法。这对于离策略算法（如SAC）至关重要，因为回放缓冲区中的数据对于算法的稳定学习非常重要。

### 独立于模型保存和加载策略

```python
policy = model.policy
policy.save("sac_policy_pendulum")
saved_policy = MlpPolicy.load("sac_policy_pendulum")
```

你可以单独保存和加载策略，这对于在不需要完整模型的情况下使用训练好的策略非常有用。例如，当你只需要在环境中执行策略而不继续训练时。

### 评估策略性能

```python
mean_reward, std_reward = evaluate_policy(policy, env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")
```

使用`evaluate_policy`函数可以评估策略在给定环境中的性能。这里，`n_eval_episodes=10`指定了评估期间的剧情数，`deterministic=True`表示在评估时采用确定性的动作选择。

### 注意事项

- 当你计划在加载模型后继续训练时，推荐加载回放缓冲区，以确保学习的稳定性。
- 如果创建了新的环境或需要重置学习计时，需要在`learn`函数中传入`reset_num_timesteps=True`。
- 单独保存策略可以用于部署场景，其中模型的其他组件（如回放缓冲区）不是必需的。

通过这种方式，你可以更灵活地处理训练好的强化学习模型和策略，根据需要保存、加载和部署它们。

## Accessing and modifying model parameters

In [ ]:
# 访问和修改模型参数

from typing import Dict

import gymnasium as gym
import numpy as np
import torch as th

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy


def mutate(params: Dict[str, th.Tensor]) -> Dict[str, th.Tensor]:
    """Mutate parameters by adding normal noise to them"""
    return dict((name, param + th.randn_like(param)) for name, param in params.items())


# Create policy with a small network
model = A2C(
    "MlpPolicy",
    "CartPole-v1",
    ent_coef=0.0,
    policy_kwargs={"net_arch": [32]},
    seed=0,
    learning_rate=0.05,
)

# Use traditional actor-critic policy gradient updates to
# find good initial parameters
model.learn(total_timesteps=10_000)

# Include only variables with "policy", "action" (policy) or "shared_net" (shared layers)
# in their name: only these ones affect the action.
# NOTE: you can retrieve those parameters using model.get_parameters() too
mean_params = dict(
    (key, value)
    for key, value in model.policy.state_dict().items()
    if ("policy" in key or "shared_net" in key or "action" in key)
)

# population size of 50 invdiduals
pop_size = 50
# Keep top 10%
n_elite = pop_size // 10
# Retrieve the environment
vec_env = model.get_env()

for iteration in range(10):
    # Create population of candidates and evaluate them
    population = []
    for population_i in range(pop_size):
        candidate = mutate(mean_params)
        # Load new policy parameters to agent.
        # Tell function that it should only update parameters
        # we give it (policy parameters)
        model.policy.load_state_dict(candidate, strict=False)
        # Evaluate the candidate
        fitness, _ = evaluate_policy(model, vec_env)
        population.append((candidate, fitness))
    # Take top 10% and use average over their parameters as next mean parameter
    top_candidates = sorted(population, key=lambda x: x[1], reverse=True)[:n_elite]
    mean_params = dict(
        (
            name,
            th.stack([candidate[0][name] for candidate in top_candidates]).mean(dim=0),
        )
        for name in mean_params.keys()
    )
    mean_fitness = sum(top_candidate[1] for top_candidate in top_candidates) / n_elite
    print(f"Iteration {iteration + 1:<3} Mean top fitness: {mean_fitness:.2f}")
    print(f"Best fitness: {top_candidates[0][1]:.2f}")

这个例子演示了如何通过修改模型参数来实现进化策略（Evolution Strategy，简称ES）解决`CartPole-v1`环境问题。首先，使用A2C算法找到一组好的初始参数，然后通过在参数上添加正态分布噪声来生成候选解，并评估这些候选解的性能，最后选取表现最好的一部分候选解更新参数。这是一个简单的参数优化过程，演示了如何在Stable Baselines3中访问和修改模型参数。

### 创建模型和训练

```python
model = A2C(
    "MlpPolicy",
    "CartPole-v1",
    ent_coef=0.0,
    policy_kwargs={"net_arch": [32]},
    seed=0,
    learning_rate=0.05,
)
model.learn(total_timesteps=10_000)
```

- 使用A2C算法创建并训练模型。这里使用一个较小的网络（只有32个神经元的隐层）来快速找到一个合理的参数集合。

### 获取和修改参数

```python
mean_params = dict(
    (key, value)
    for key, value in model.policy.state_dict().items()
    if "policy" in key or "shared_net" in key or "action" in key
)

def mutate(params: Dict[str, th.Tensor]) -> Dict[str, th.Tensor]:
    return dict((name, param + th.randn_like(param)) for name, param in params.items())
```

- 从模型中获取参数并选择影响动作的参数（即名称中包含"policy"、"action"或"shared_net"的参数）。
- 定义`mutate`函数，对参数进行突变，即在每个参数上添加正态分布噪声。

### 进化策略循环

```python
for iteration in range(10):
    population = []
    for population_i in range(pop_size):
        candidate = mutate(mean_params)
        model.policy.load_state_dict(candidate, strict=False)
        fitness, _ = evaluate_policy(model, vec_env)
        population.append((candidate, fitness))
    top_candidates = sorted(population, key=lambda x: x[1], reverse=True)[:n_elite]
    mean_params = dict(
        (name, th.stack([candidate[0][name] for candidate in top_candidates]).mean(dim=0))
        for name in mean_params.keys()
    )
```

- 对于每次迭代，通过在当前的平均参数上添加噪声来创建一组候选解，然后评估这些候选解的性能。
- 选择表现最好的一部分候选解，并计算它们参数的平均值作为下一代的平均参数。
- 这个过程重复进行，直到完成指定的迭代次数。

### 结果和输出

```python
print(f"Iteration {iteration + 1:<3} Mean top fitness: {mean_fitness:.2f}")
print(f"Best fitness: {top_candidates[0][1]:.2f}")
```

- 在每次迭代后，输出当前迭代的平均最优适应度和最佳适应度。

这个例子展示了如何在不借助于传统的基于梯度的优化方法的情况下，通过直接操作模型参数来改进RL模型的性能。进化策略是一种强大的全局优化技术，特别适合于处理高维、非凸或者梯度信息不可用的优化问题。

## SB3 and ProcgenEnv

In [ ]:
from procgen import ProcgenEnv

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecExtractDictObs, VecMonitor

# ProcgenEnv is already vectorized
venv = ProcgenEnv(num_envs=2, env_name="starpilot")

# To use only part of the observation:
# venv = VecExtractDictObs(venv, "rgb")

# Wrap with a VecMonitor to collect stats and avoid errors
venv = VecMonitor(venv=venv)

model = PPO("MultiInputPolicy", venv, verbose=1)
model.learn(10_000)

这个例子展示了如何使用Stable Baselines3（SB3）与Procgen环境一起进行训练。Procgen提供了一系列用于强化学习研究的程序生成环境，这些环境支持向量化操作，使得可以同时运行多个环境实例以加速训练过程。

### 使用Procgen环境

```python
from procgen import ProcgenEnv
venv = ProcgenEnv(num_envs=2, env_name="starpilot")
```

- `ProcgenEnv`是Procgen库中用于创建环境的函数。`num_envs=2`指定了要同时运行的环境实例数量，`env_name="starpilot"`指定了要使用的环境名称。

### 包装环境

虽然Procgen环境已经是向量化的，但为了与SB3兼容并收集训练统计信息，需要使用`VecMonitor`包装器对环境进行包装。

```python
from stable_baselines3.common.vec_env import VecMonitor
venv = VecMonitor(venv=venv)
```

- `VecMonitor`包装器用于监视向量化环境的性能，并记录统计数据，如奖励和长度等信息。

### 可选：提取特定的观察部分

如果你只对环境观察的一部分感兴趣，可以使用`VecExtractDictObs`包装器来提取特定的观察字段。

```python
from stable_baselines3.common.vec_env import VecExtractDictObs
# venv = VecExtractDictObs(venv, "rgb")
```

- 这行代码被注释掉了，但如果你取消注释，它将配置环境仅使用名为"rgb"的观察字段。这对于处理复杂观察空间特别有用。

### 训练模型

```python
from stable_baselines3 import PPO
model = PPO("MultiInputPolicy", venv, verbose=1)
model.learn(10_000)
```

- 使用PPO算法和多输入策略（`MultiInputPolicy`）对模型进行实例化并开始训练。`verbose=1`参数用于打印训练进度和统计信息。
- 在这里，`MultiInputPolicy`是因为Procgen环境的观察可能是多模态的（例如，同时包含图像和向量数据），SB3可以处理这种情况。

### 结论

这个例子展示了如何将SB3与Procgen环境结合使用，包括如何处理向量化环境和收集训练过程中的统计数据。通过这种方式，你可以利用SB3的强大功能和Procgen环境的多样性，进行高效的强化学习研究和开发。

## SB3 with EnvPool or Isaac Gym

EnvPool和Isaac Gym是两个用于加速强化学习训练的工具，它们通过提供向量化的环境实现来实现这一点，类似于Procgen环境。这意味着它们可以同时运行多个环境实例，从而大大加速训练过程。为了将这些工具与Stable Baselines3（SB3）结合使用，需要通过特定的`VecEnvWrapper`来包装环境，这个包装器将为SB3预处理数据。

### 使用EnvPool或Isaac Gym的关键步骤包括：

1. **安装EnvPool或Isaac Gym**：首先需要确保你已经安装了EnvPool或Isaac Gym。这通常可以通过`pip`命令来完成，具体安装步骤请参考各自的官方文档。

2. **创建向量化环境**：使用EnvPool或Isaac Gym创建向量化环境。这些工具的API允许你指定要同时运行的环境实例数量，从而实现并行化。

3. **包装环境**：为了使得这些环境与SB3兼容，你需要使用特定的`VecEnvWrapper`来包装环境。这些包装器负责预处理环境的观察、奖励等数据，使其符合SB3的数据格式要求。

   在[SB3的issue #772](https://github.com/DLR-RM/stable-baselines3/issues/772)中，提供了EnvPool和Isaac Gym的`VecEnvWrapper`链接。这些包装器是由社区成员贡献的，可以帮助你轻松地将这些工具与SB3结合使用。

4. **训练模型**：环境准备好之后，就可以使用SB3的算法（如PPO、SAC等）来训练模型了。这个过程与使用标准Gym环境类似，只需要将包装后的环境传递给模型的构造函数即可。

### 示例代码结构（假设代码）：

```python
from stable_baselines3 import PPO
from envpool import GymEnvPool
from sb3_contrib.common.wrappers import EnvPoolVecEnvWrapper

# 创建EnvPool环境
env = GymEnvPool('CartPole-v1', num_envs=4)

# 使用EnvPool的VecEnvWrapper包装环境
env = EnvPoolVecEnvWrapper(env)

# 使用SB3训练模型
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)
```

**注意**：上面的代码是一个示例性的代码结构，具体实现可能会根据EnvPool或Isaac Gym的实际API有所不同。你需要参考具体的`VecEnvWrapper`实现和相关文档来正确集成这些工具。

结合使用EnvPool、Isaac Gym和SB3可以显著加速强化学习训练过程，特别是对于那些需要大量计算资源和时间的复杂任务。通过利用这些工具提供的向量化环境和并行处理能力，可以在更短的时间内探索更广泛的策略空间，从而提高模型的性能和鲁棒性。

## Record a Video

In [ ]:
import gymnasium as gym
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

env_id = "CartPole-v1"
video_folder = "logs/videos/"
video_length = 100

vec_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])

obs = vec_env.reset()

# Record the video starting at the first step
vec_env = VecVideoRecorder(vec_env, video_folder,
                       record_video_trigger=lambda x: x == 0, video_length=video_length,
                       name_prefix=f"random-agent-{env_id}")

vec_env.reset()
for _ in range(video_length + 1):
  action = [vec_env.action_space.sample()]
  obs, _, _, _ = vec_env.step(action)
# Save the video
vec_env.close()

这个例子展示了如何使用Stable Baselines3和gymnasium库录制一个使用随机智能体在CartPole-v1环境中的视频。为了成功录制视频，你的机器上需要安装`ffmpeg`或`avconv`。

### 步骤说明

1. **设置环境和视频参数**：

首先，定义环境ID、视频存储的文件夹以及视频的长度（以环境步骤计）。

```python
env_id = "CartPole-v1"
video_folder = "logs/videos/"
video_length = 100
```

2. **创建向量化环境**：

使用`DummyVecEnv`创建一个向量化环境，这是为了与`VecVideoRecorder`兼容。这里，通过`lambda`函数创建了一个`CartPole-v1`环境实例，并设置`render_mode="rgb_array"`以允许环境渲染返回RGB数组。

```python
vec_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
```

3. **设置视频录制器**：

接下来，使用`VecVideoRecorder`包装刚刚创建的向量化环境，设置视频录制的相关参数。`record_video_trigger=lambda x: x == 0`表示视频录制将从第一步开始。`video_length=video_length`指定了视频的长度。`name_prefix`定义了视频文件的前缀。

```python
vec_env = VecVideoRecorder(vec_env, video_folder,
                           record_video_trigger=lambda x: x == 0, video_length=video_length,
                           name_prefix=f"random-agent-{env_id}")
```

4. **运行环境并录制视频**：

然后，重置环境并通过循环执行随机动作来运行环境，环境的每一步都会被录制下来。

```python
vec_env.reset()
for _ in range(video_length + 1):
  action = [vec_env.action_space.sample()]
  obs, _, _, _ = vec_env.step(action)
```

5. **保存视频**：

最后，调用`vec_env.close()`来结束视频录制并保存视频文件到之前指定的文件夹中。

```python
# Save the video
vec_env.close()
```

### 注意事项

- 请确保你的机器上已安装`ffmpeg`或`avconv`，这是录制视频的必要条件。
- 视频将被保存在指定的文件夹中，文件名将以指定的前缀开始，后跟环境ID和一些其他信息。
- 这种方法可以用于录制任何gym环境的视频，对于展示智能体的表现、调试或创建展示材料非常有用。

## Bonus: Make a GIF of a Trained Agent

In [ ]:
import imageio
import numpy as np

from stable_baselines3 import A2C

model = A2C("MlpPolicy", "LunarLander-v2").learn(100_000)

images = []
obs = model.env.reset()
img = model.env.render(mode="rgb_array")
for i in range(350):
    images.append(img)
    action, _ = model.predict(obs)
    obs, _, _ ,_ = model.env.step(action)
    img = model.env.render(mode="rgb_array")

imageio.mimsave("lander_a2c.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

这个示例展示了如何使用Stable Baselines3训练一个A2C模型，并将训练好的智能体在LunarLander-v2环境中的表现制作成GIF。以下是详细步骤：

### 训练模型

首先，使用A2C算法和多层感知器策略（`MlpPolicy`）在`LunarLander-v2`环境上训练模型。

```python
from stable_baselines3 import A2C

model = A2C("MlpPolicy", "LunarLander-v2").learn(100_000)
```

### 收集图像

然后，通过执行智能体在环境中的动作，并在每一步捕获环境的RGB图像，收集图像序列。

```python
import numpy as np

images = []
obs = model.env.reset()
img = model.env.render(mode="rgb_array")
for i in range(350):
    images.append(img)
    action, _ = model.predict(obs)
    obs, _, _, _ = model.env.step(action)
    img = model.env.render(mode="rgb_array")
```

- 在这里，使用了`model.env.render(mode="rgb_array")`来获取当前环境的RGB图像，并将其添加到图像列表中。
- 循环执行350步操作，这个数字可以根据你的需求调整，以获取足够长的动作序列。

### 制作GIF

最后，使用`imageio.mimsave`将图像序列保存为GIF文件。为了减少GIF的大小，这里每两帧选取一帧进行保存。

```python
import imageio

imageio.mimsave("lander_a2c.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)
```

- `[np.array(img) for i, img in enumerate(images) if i%2 == 0]`这部分代码是在说，从图像序列中选取每两帧中的一帧。
- `fps=29`指定了GIF的帧率，你可以根据需要调整这个值。

### 注意事项

- 请确保你的环境安装了`imageio`库，如果没有，可以通过`pip install imageio`安装。
- 这个方法可以用于生成任何gym环境智能体表现的GIF，非常适合用于演示或者分析智能体的行为。
- 制作GIF是一个很好的方式来可视化和分享智能体的学习成果。